In [2]:
#### THIS FILE IS DONE AND ON GIT AS SRC/DATA_PIPELINE.PY


# core imports
from datetime import datetime
import numpy as np
import pandas as pd
from time import sleep


# Aplhavantage
from alpha_vantage.techindicators import TechIndicators
from alpha_vantage.timeseries import TimeSeries


In [3]:
# tickers = ['AAPL','ACN','ADBE','ADI','ADP','ADSK','AKAM','AMAT','AMD','ANET','ANSS','APH','AVGO','BR','CDNS','CDW','CRM','CSCO','CTSH','CTXS','DXC','ENPH','FFIV','FIS','FISV','FLIR','FLT','FTNT','GLW','GPN','HPE','HPQ','IBM','INTC','INTU','IPGP','IT','JKHY','JNPR','KEYS','KLAC','LRCX','MA','MCHP','MPWR','MSFT','MSI','MU','MXIM','NLOK','NOW','NTAP','NVDA','NXPI','ORCL','PAYC','PAYX','PYPL','QCOM','QRVO','SNPS','STX','SWKS','TEL','TER','TRMB','TXN','TYL','V','VRSN','WDC','WU','XLNX','ZBRA']
tickers = ['AAPL','ACN']

In [14]:
def API(ticker):
    API_key = 'CSMN0LYTQ5UYMVUT'
    API_URL = "https://www.alphavantage.co/query"
    
    
    ts = TimeSeries(key=API_key, output_format='pandas')
    price = ts.get_daily_adjusted(ticker, outputsize='full')[0]
    
    
    
    ti = TechIndicators(key=API_key, output_format='pandas')
    
    macd, meta_data = ti.get_macd(symbol=ticker,interval='daily')
    bbands, meta_data = ti.get_bbands(symbol=ticker,interval='daily')
    rsi,meta_data = ti.get_rsi(symbol=ticker,interval='daily',time_period=15)
    sma5, meta_data = ti.get_sma(symbol=ticker,time_period=5)
    sma15, meta_data = ti.get_sma(symbol=ticker,time_period=15)
    roc, meta_data = ti.get_roc(symbol=ticker,series_type='high',interval='daily')
    cci, meta_data = ti.get_cci(symbol=ticker,time_period=60,interval='daily')
    dx, meta_data = ti.get_dx(symbol=ticker,time_period=60,interval='daily')
    
    data = pd.concat([price, macd, bbands, rsi, sma5, sma15, roc, cci, dx], axis=1)
    data = data.dropna()
    # data['ticker'] = ticker
    return data
    
    
    
    
    

In [20]:
def crossover(data):
    data['crossover'] = data['MACD'] > data['MACD_Signal']
    return data
def cutoff(data):
    return data[data.index>=datetime(2015, 1, 1)]
def dropcols(data):
    data = data.drop(columns=['4. close', '7. dividend amount', '8. split coefficient'])
    return data
def feature_names(data):
    data = data.rename(columns={'1. open':'open', '2. high':'high', '3. low':'low', '5. adjusted close':'Close', '6. volume':'vol'})
    return data

In [21]:
def preprocess_data(tickers):
    for ticker in tickers:
        data = API(ticker)
        data = crossover(data)
        data = cutoff(data)
        data = dropcols(data)
        data.fillna(method='bfill',inplace=True)
        data = feature_names(data)
        data.to_csv(f'data/{ticker}.csv', index=True)
    sleep(7)    
    pass

In [22]:
preprocess_data(tickers)

In [23]:
df = pd.read_csv('data/AAPL.csv')

In [24]:
df.head()

,date,open,high,low,Close,vol,MACD_Signal,MACD_Hist,MACD,Real Upper Band,Real Lower Band,Real Middle Band,RSI,SMA,SMA.1,ROC,CCI,DX,crossover
0,2015-01-02,111.39,111.44,107.350,24.859222,53204626.0,0.0331,-0.0662,-0.0331,26.4382,24.4203,25.4293,43.3753,25.4722,25.2811,-4.2200,-1.2375,1.0230,False
1,2015-01-05,108.29,108.65,105.410,24.158898,64285491.0,0.0005,-0.1305,-0.1300,26.4006,24.2478,25.3242,37.1471,25.1203,25.1997,-7.2952,-39.5183,4.1662,False
2,2015-01-06,106.54,107.43,104.630,24.161172,65797116.0,-0.0405,-0.1638,-0.2043,26.3446,24.1051,25.2248,37.1785,24.7724,25.1471,-7.4518,-51.0223,5.3946,False
3,2015-01-07,107.20,108.20,106.695,24.499965,40105934.0,-0.0790,-0.1541,-0.2332,26.3233,24.0206,25.1720,41.8158,24.5554,25.1399,-5.6242,-32.8844,4.0556,False
4,2015-01-08,109.23,112.15,108.700,25.441309,59364547.0,-0.0988,-0.0792,-0.1780,26.2498,24.0434,25.1466,52.2985,24.6241,25.2179,-1.8810,14.4123,2.4124,False
